In [1]:
import os
from kerashypetune import KerasGridSearch
from keras.optimizers import SGD

from lib.data_loader import DataLoader
from lib.image import ImageDataGenerator
from lib.resnet_model import Resnet3DBuilder

Using TensorFlow backend.


In [2]:
root_directory = r'<path>\dataset'
labels_csv_path = os.path.join(root_directory, 'labels_extracted.csv')
train_csv_path = os.path.join(root_directory, 'train_extracted2.csv')
val_csv_path = os.path.join(root_directory, 'validation_extracted2.csv')
train_path = os.path.join(root_directory, 'Train')
val_path = os.path.join(root_directory, 'Validation')

data = DataLoader(labels_csv_path, train_csv_path, val_csv_path)

In [3]:
WIDTH = 96
HEIGHT = 64
TARGET_SIZE = (HEIGHT, WIDTH)
BATCH_SIZE = 32
N_FRAMES = 16
SKIP = 1
INPUT_SHAPE = (N_FRAMES,) + TARGET_SIZE + (3,)
N_CLASSES = 10
EPOCHS = 1
WORKERS = 2

param_grid = {
    'drop_rate': [0.2, 0.5],
    'learning_rate': [0.001, 0.01]
}

In [4]:
def define_model(params):

    model = Resnet3DBuilder.build_resnet_101(
        input_shape=INPUT_SHAPE, 
        num_outputs=N_CLASSES,
        drop_rate=params['drop_rate']
    )

    optimizer = SGD(learning_rate=params['learning_rate'])
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [5]:
image_datagen = ImageDataGenerator()

train_generator = image_datagen.flow_video_from_dataframe(
    dataframe=data.train_df, 
    directory=train_path, 
    path_classes=labels_csv_path, 
    x_col='video_id', 
    y_col='label', 
    target_size=TARGET_SIZE, 
    class_mode='categorical',
    color_mode='rgb',
    batch_size=BATCH_SIZE, 
    shuffle=True,
    seed=42,
    nb_frames=N_FRAMES, 
    skip=SKIP
)

validation_generator = image_datagen.flow_video_from_dataframe(
    dataframe=data.val_df, 
    directory=val_path, 
    path_classes=labels_csv_path, 
    x_col='video_id', 
    y_col='label', 
    target_size=TARGET_SIZE, 
    class_mode='categorical',
    color_mode='rgb',
    batch_size=BATCH_SIZE, 
    shuffle=True,
    seed=42,
    nb_frames=N_FRAMES, 
    skip=SKIP
)

Found 5500 video folders belonging to 10 classes.
Found 1100 video folders belonging to 10 classes.


In [7]:
kgs = KerasGridSearch(
    hypermodel=define_model, 
    param_grid=param_grid,
    monitor='val_accuracy', 
    greater_is_better=True, 
    tuner_verbose=1
)

n_sample_train = len(data.train_df)
n_sample_val = len(data.val_df)

kgs.search(
    x=train_generator, 
    steps_per_epoch=n_sample_train/BATCH_SIZE,
    validation_data=validation_generator, 
    validation_steps=n_sample_val/BATCH_SIZE,
    epochs=1, 
    workers=WORKERS,
    verbose=1
)


4 trials detected for ('drop_rate', 'learning_rate')

***** (1/4) *****
Search({'drop_rate': 0.2, 'learning_rate': 0.01})
Epoch 1/1
172/171 [==============================] - 5381s 31s/step - loss: 12.9015 - accuracy: 0.1029 - val_loss: 12.9287 - val_accuracy: 0.1073
SCORE: 0.10727 at epoch 1

***** (2/4) *****
Search({'drop_rate': 0.2, 'learning_rate': 0.001})
Epoch 1/1
172/171 [==============================] - 5477s 32s/step - loss: 12.8982 - accuracy: 0.1085 - val_loss: 12.8295 - val_accuracy: 0.1045
SCORE: 0.10455 at epoch 1

***** (3/4) *****
Search({'drop_rate': 0.5, 'learning_rate': 0.01})
Epoch 1/1
172/171 [==============================] - 6274s 36s/step - loss: 12.9147 - accuracy: 0.1033 - val_loss: 12.8536 - val_accuracy: 0.1145
SCORE: 0.11455 at epoch 1

***** (4/4) *****
Search({'drop_rate': 0.5, 'learning_rate': 0.001})
Epoch 1/1
172/171 [==============================] - 8765s 51s/step - loss: 12.9106 - accuracy: 0.0949 - val_loss: 12.8355 - val_accuracy: 0.0900
SCORE:

In [9]:
kgs.best_params

{'drop_rate': 0.5,
 'learning_rate': 0.01,
 'epochs': 1,
 'steps_per_epoch': 171.875,
 'batch_size': None}